In [1]:
# NLP
# import nltk
# nltk.download()

In [2]:
import re
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from joblib import load
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [3]:
from text_cleaning import clean_up

In [4]:
# load products
df = pd.read_json('products.json', orient='records')
df.head()

,product_code,product_name,product_price,product_category,product_description,product_more_info,product_image
0,00007,Contorno De Ojos En Gel Hidratante,200,faciales,Nuestros Expertos en Cosmética Vegetal han sel...,EDULIS: Nuestros Investigadores en Cosmética V...,https://www.yvesrocher.com.mx/Areas/YR/images/...
1,01176,Crema Facial Exfoliante Aclarante,290,faciales,Exfoliación suave que elimina a diario las cél...,LUPINO BLANCO Y REGALIZ Nuestros expertos en I...,https://www.yvesrocher.com.mx/Areas/YR/images/...
2,01340,Maxi Gel Facial Limpiador Hidratante,340,faciales,Nuestros Expertos en Cosmética Vegetal han sel...,EDULIS: Nuestros Investigadores en Cosmética V...,https://www.yvesrocher.com.mx/Areas/YR/images/...
3,06043,Mascarilla Facial Nocturna Aclarante,359,faciales,"Su piel es suave, hidratada y radiante después...",LUPINO BLANCO Y REGALIZ Nuestros expertos en I...,https://www.yvesrocher.com.mx/Areas/YR/images/...
4,11783,Crema Facial Suavizante Día/noche,175,faciales,"Hidrata,suaviza y protege la piel. Su plus:\nT...","MANZANILLA En 1964, Yves Rocher imaginó por pr...",https://www.yvesrocher.com.mx/Areas/YR/images/...


In [5]:
# load clean text
text_series = load('text_series.joblib')
text_series.head()

0    contorn ojos gel hidrat facial expert cosmet v...
1    crem facial exfoli aclar facial exfoli suav el...
2    maxi gel facial limpiador hidrat facial expert...
3    mascarill facial nocturn aclar facial piel sua...
4    crem facial suaviz dia noch facial hidrat suav...
dtype: object

In [6]:
# Define a TF-IDF Vectorizer Object
vectorizer = TfidfVectorizer(
    analyzer='word',
    ngram_range=(1, 2),
    min_df=0.003
)

In [7]:
#Construct a reverse map of indices and product codes
indices = pd.Series(df.index, index=df.product_code).drop_duplicates()

In [8]:
# Function that takes a description as input and outputs most similar product
def get_recommendation(description):
    description = clean_up(description)
    tmp_series = pd.concat([
        text_series,
        pd.Series([description])
    ], ignore_index=True)
    
    #Construct the required TF-IDF matrix by fitting and transforming the data
    tfidf_matrix = vectorizer.fit_transform(tmp_series)

    # Compute the cosine similarity matrix
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    
    # Set the index of product description
    idx = -1
    
    # Get the pairwsie similarity scores of all products with that product
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Sort the products based on the scores
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse=True)
    
    # Get the score of the 10 most similar products
    sim_scores = sim_scores[1:11]
    
    # Get the product indices
    product_indices = [i[0] for i in sim_scores]
    
    # Return  the top most similar products
    return df.iloc[product_indices]

In [9]:
my_description = '''
aroma fresco
'''

In [10]:
recommendation = get_recommendation(my_description)
recommendation

,product_code,product_name,product_price,product_category,product_description,product_more_info,product_image
99,40291,Jumbo Shampoo Brillo Espectacular,249,capilares,El Shampoo Brillo Espectácular limpia suavemen...,EXTRACTO DE LIMÓN Y VITAMINA VEGETAL Aportan u...,https://www.yvesrocher.com.mx/Areas/YR/images/...
102,42879,Shampoo Brillo Espectacular,109,capilares,El Shampoo Brillo Espectácular limpia suavemen...,EXTRACTO DE LIMÓN Y VITAMINA VEGETAL Aportan u...,https://www.yvesrocher.com.mx/Areas/YR/images/...
103,43680,Eco Shampoo Concentrado Brillo Espectacular,89,capilares,"""CABELLOS SUAVES Y BRILLANTES\nLa fórmula conc...","""EXTRACTO DE LIMÓN Y VITAMINA VEGETAL Aportan ...",https://www.yvesrocher.com.mx/Areas/YR/images/...
506,44312,Agua De Perfume Tropicale Tentation,439,fragancias,El Agua de Perfume Tropicale Tentation es el p...,NOTAS OLFATIVAS: FLOR DORADA DE OSMANTHUS Y NA...,https://www.yvesrocher.com.mx/Areas/YR/images/...
516,47112,Mini Agua De Perfume Tropicale Tentation,359,fragancias,El Agua de Perfume Tropicale Tentation es el p...,NOTAS OLFATIVAS: FLOR DORADA DE OSMANTHUS Y NA...,https://www.yvesrocher.com.mx/Areas/YR/images/...
171,62501,Crema Para Manos Tropicale Tentation,79,corporales,Descubre tu fragancia favorita en esta suave y...,NOTAS OLFATIVAS: FLOR DORADA DE OSMANTHUS Y NA...,https://www.yvesrocher.com.mx/Areas/YR/images/...
72,67014,Contorno De Ojos Efecto Lifting,285,faciales,"Este gel de textura fresca hidrata, reduce las...","BÚGULA: La Ajuga Reptans, también conocida com...",https://www.yvesrocher.com.mx/Areas/YR/images/...
497,37086,Agua De Tocador Comme Une Evidence Homme,349,fragancias,Nuestro Vetiver haitiano garantiza un sumistro...,"NOTAS OLFATIVAS: Vetiver, Gaiac, Bergamota, Pi...",https://www.yvesrocher.com.mx/Areas/YR/images/...
162,55561,Crema Corporal Fluida Mango Y Cilantro,149,corporales,Para el placer de un tratamiento corporal ener...,MANGO Y CILANTRO Yves Rocher ha seleccionado e...,https://www.yvesrocher.com.mx/Areas/YR/images/...
151,31253,Maxi Crema Corporal Fluida Mango Y Cilantro,119,corporales,Para el placer de un tratamiento corporal ener...,MANGO Y CILANTRO Yves Rocher ha seleccionado e...,https://www.yvesrocher.com.mx/Areas/YR/images/...


In [11]:
recommendation.to_json('recommendation.json', orient='records')